# Home Credit Default Risk

Outline:
* Load the data
* Join tables with Polars - a DataFrame library implemented in Rust language, very fast and memory efficient.  
* Create features
* Train models
* Create a submission table

## Load the data
Data loading and polars code credit: https://www.kaggle.com/code/jetakow/home-credit-2024-starter-notebook

In [17]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 

dataPath = "./data/"

In [18]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

    return df

In [19]:
def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

## Data information
* basetable:
* static:
* static_cb:
 

In [21]:
train_basetable = pl.read_csv(dataPath + "csv_files/train/train_base.csv")
train_static = pl.concat(    [
        pl.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(set_table_dtypes)
train_person_1 = pl.read_csv(dataPath + "csv_files/train/train_person_1.csv").pipe(set_table_dtypes) 
train_credit_bureau_b_2 = pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_b_2.csv").pipe(set_table_dtypes) 

In [22]:
test_basetable = pl.read_csv(dataPath + "csv_files/test/test_base.csv")
test_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/test/test_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_1.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_2.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv").pipe(set_table_dtypes)
test_person_1 = pl.read_csv(dataPath + "csv_files/test/test_person_1.csv").pipe(set_table_dtypes) 
test_credit_bureau_b_2 = pl.read_csv(dataPath + "csv_files/test/test_credit_bureau_b_2.csv").pipe(set_table_dtypes) 

In [23]:
train_basetable.head()

case_id,date_decision,MONTH,WEEK_NUM,target
i64,str,i64,i64,i64
0,"""2019-01-03""",201901,0,0
1,"""2019-01-03""",201901,0,0
2,"""2019-01-04""",201901,0,0
3,"""2019-01-03""",201901,0,0
4,"""2019-01-04""",201901,0,1


In [24]:
train_static.head()

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,…,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,payvacationpostpone_4187118D,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,previouscontdistrict_112M,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,…,str,str,f64,f64,str,f64,str,f64,str,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64
0,null,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,-1,0,0,-1,null,null,null,null,null,24.0,0.0,0.0,null,0,null,0.0,0.0,null,null,0.0,0.0,null,0,-1,-1
1,null,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0.0""",3.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,-1,0,0,-1,null,null,null,null,null,18.0,0.0,0.0,null,0,null,0.0,0.0,null,null,0.0,0.0,null,0,-1,-1
2,null,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0,0,0,-1,null,null,null,null,null,36.0,0.0,0.0,null,0,null,0.0,0.0,null,null,0.0,0.0,null,0,0,-1
3,null,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,null,null,null,null,null,null,null,null,null,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,1.0,0,0,0,-1,null,null,null,null,null,12.0,0.0,0.0,null,0,null,1.0,1.0,null,null,0.0,0.0,null,0,0,-1
4,null,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0,0,0,-1,null,null,null,null,null,24.0,0.0,0.0,null,0,null,0.0,0.0,null,null,0.0,0.0,null,0,0,-1


In [25]:
test_static_cb.head()

case_id,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
i64,str,str,str,str,f64,str,str,f64,f64,f64,f64,f64,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,f64,f64,f64,f64,str,f64,str,str,f64,str,str,str,str,str,str,f64,f64
57543,null,null,null,null,151364.0,"""1980-11-01""",null,2.0,4.0,1.0,8.0,2.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",4.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9.0,"""38c061ee""","""a55475b1""",8.0,null,null,null,null,null,null,null,null,null,null,null,"""2021-05-28""",null,null,2.0,3.0
57549,null,null,"""2018-05-06""",null,1.5631e6,"""1959-11-01""",null,6.0,9.0,3.0,12.0,4.0,"""2fc785b2""","""39a0853f""","""a55475b1""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5.0,"""a7fcb6e5""","""a55475b1""",12.0,null,null,26815.6,null,14.0,null,null,null,null,null,null,"""2022-01-31""",null,null,8.0,2.0
57551,null,null,null,null,2926195.3,"""1982-05-01""",null,1.0,3.0,1.0,4.0,1.0,"""2fc785b2""","""6b2ae0fa""","""a55475b1""",3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.0,"""3439d993""","""a55475b1""",4.0,null,null,null,null,null,null,null,null,null,null,null,"""2020-12-11""",null,null,5.0,5.0
57552,null,null,"""2018-11-18""",null,747031.73,"""1955-11-01""",null,2.0,2.0,0.0,5.0,0.0,"""2fc785b2""","""a55475b1""","""a55475b1""",3.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.0,"""a55475b1""","""a55475b1""",5.0,null,null,23402.8,null,14.0,null,null,null,null,null,null,"""2020-12-11""",null,null,7.0,1.0
57569,null,null,"""2011-10-14""",null,null,"""1949-09-01""",null,4.0,4.0,1.0,4.0,4.0,"""2fc785b2""","""717ddd49""","""a55475b1""",0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,"""3439d993""","""a55475b1""",4.0,null,null,17333.6,null,14.0,null,null,null,null,null,null,"""2022-01-03""",null,null,1.0,3.0


In [26]:
test_person_1.head()

case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,familystate_447L,gender_992L,housetype_905L,housingtype_772L,incometype_1044T,isreference_387L,language1_981M,mainoccupationinc_384A,maritalst_703L,num_group1,personindex_1023L,persontype_1072L,persontype_792L,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
i64,str,str,f64,str,bool,bool,str,str,str,str,str,str,str,str,str,str,str,str,bool,str,f64,str,i64,f64,f64,f64,str,str,str,str,bool,str,str,bool,str,str
57543,"""1980-11-01""",null,null,"""P107_155_111""",false,false,"""P91_47_168""","""P33_146_175""",null,null,null,"""a55475b1""","""a55475b1""","""SINGLE""",null,null,null,"""SALARIED_GOVT""",null,"""a55475b1""",34000.0,null,0,0.0,1.0,1.0,"""P107_155_111""","""P91_47_168""",null,null,null,"""CL""",null,false,"""F""","""PRIMARY_MOBILE…"
57543,null,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,null,1,1.0,5.0,5.0,"""a55475b1""","""a55475b1""","""SIBLING""","""SIBLING""",false,"""PE""",null,null,null,"""PHONE"""
57549,"""1959-11-01""",null,null,"""a55475b1""",null,null,"""a55475b1""","""P106_81_188""",null,null,null,"""a55475b1""","""a55475b1""","""SINGLE""",null,null,null,"""RETIRED_PENSIO…",null,"""a55475b1""",49800.0,null,0,0.0,1.0,1.0,"""P121_131_159""","""P111_112_180""",null,null,null,"""CL""",null,true,"""F""","""PRIMARY_MOBILE…"
57549,null,null,null,"""a55475b1""",null,null,"""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,null,"""a55475b1""",null,null,1,1.0,5.0,5.0,"""a55475b1""","""a55475b1""","""COLLEAGUE""","""COLLEAGUE""",false,"""PE""",null,null,null,"""PHONE"""
57551,"""1982-05-01""",null,null,"""P11_36_178""",false,false,"""P97_107_128""","""a55475b1""","""2002-01-08""",null,null,"""a55475b1""","""a55475b1""",null,null,null,null,"""SALARIED_GOVT""",null,"""P10_39_147""",59600.0,null,0,0.0,1.0,1.0,"""P11_36_178""","""P97_107_128""",null,null,null,"""CL""",null,false,"""F""","""PRIMARY_MOBILE…"


In [27]:
train_credit_bureau_b_2.head()

case_id,num_group1,num_group2,pmts_date_1107D,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
i64,i64,i64,i64,f64,f64
467,0,0,43417,null,null
467,0,1,43447,null,null
467,1,0,43447,null,null
467,2,0,42656,0.0,0.0
467,2,1,42687,0.0,0.0


In [28]:
train_basetable.shape, train_static.shape, train_static_cb.shape, train_person_1.shape, train_credit_bureau_b_2.shape

((1526659, 5), (1526659, 168), (1500476, 53), (2973991, 37), (1286755, 6))

In [30]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or 
# also num_group2 column.
train_person_1_feats_1 = train_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").cast(int).max().alias("mainoccupationinc_384A_any_selfemployed")
)

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

# Here we have num_goup1 and num_group2, so we need to aggregate again.
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)


ComputeError: cannot compare string with numeric data

# Select columns to join

In [ ]:
selected_static_cols_test = []
selected_static_cols_test2 = []
cols1 = []
cols2 = []
for col in train_static.columns:
    cols1.append(col[-1])
    selected_static_cols_test.append(col)

for col in train_static_cb.columns:
    cols2.append(col[-1])
    selected_static_cols_test2.append(col)
# print(np.unique(cols1))

# print(np.unique(cols2))
print(selected_static_cols_test)
print(selected_static_cols_test2)
print(train_static_cb.columns)

In [ ]:
# We will process in this examples only A-type and M-type columns, so we need to select them.
selected_static_cols = []
for col in train_static.columns:

    if col[-1] in ['A', 'D', 'L', 'M']: # ['A' 'D' 'L' 'M' 'P' 'd']
        selected_static_cols.append(col)
print(selected_static_cols)

selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ['A', 'D' ,'L', 'M']: # ['A' 'D' 'L' 'M' 'T' 'd']
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)

# Join all tables together.
data = train_basetable.join(
    train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    train_person_1_feats_1, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
).join(
    train_credit_bureau_b_2_feats, how="left", on="case_id"
)

In [ ]:
test_person_1_feats_1 = test_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

test_person_1_feats_2 = test_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

test_credit_bureau_b_2_feats = test_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

data_submission = test_basetable.join(test_static.select(["case_id"]+selected_static_cols), how="left", on="case_id").join(    test_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id").join(    test_person_1_feats_1, how="left", on="case_id").join(    test_person_1_feats_2, how="left", on="case_id").join(    test_credit_bureau_b_2_feats, how="left", on="case_id")

In [ ]:
train_static_cb

In [ ]:
from  vectorization import *
#from data_reduction import *
compiled = pd.read_csv("W:/Erdos/Project/home_credit/data/csv_files/master_data_file.csv")

In [ ]:
# vectorize_dataframe(train_static_cb.to_pandas())
# vectorize(train_static_cb.to_pandas())
# vectorize_dataframe_for_nn(train_static_cb.to_pandas())


In [ ]:
# type(train_static_cb), type(train_static_cb.columns)

In [ ]:
# print(train_static.dtypes.unique())


In [ ]:
compiled.columns[:20], compiled.dtypes.unique()
# compiled.head

In [ ]:
vectorize_dataframe(compiled)

In [ ]:
from src.classification import *

In [ ]:
# labels = np.random.randint(2, size=(num_rows, 1))  # Generates 0 or 1
# print(labels.shape)
# # Create DataFrame
# df = pd.DataFrame(data, columns=['a', 'b', 'c', 'd'])
# df['e'] = labels
# print(df)

X = compiled.loc[:, compiled.columns != 'target']
y = ['target']
X_train, y_train= data_split(X, y)

print("training")
train_SimpleNN(X_train, y_train)
print("DONE")
